[![Run in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bhosmer/fold/blob/main/notebooks/transpose.ipynb)

#### Colab setup (skip if running locally)

In [2]:
!git clone https://github.com/bhosmer/fold.git
import sys
sys.path.insert(0,'/content/fold')

Cloning into 'fold'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 76 (delta 37), reused 59 (delta 27), pack-reused 4
Receiving objects: 100% (76/76), 93.46 KiB | 192.00 KiB/s, done.
Resolving deltas: 100% (37/37), done.


In [3]:
from fold import *

# Preliminaries: shape terminology

Transposition permutes an array's elements as well as its **shape**. Here we'll briefly introduce some basic shape properties and terminology used in this notebook. A more detailed introduction can be found [here](https://github.com/bhosmer/fold/blob/main/notebooks/fold_shapes.ipynb).

* fold shapes are a generalization of the standard shape metadata found in e.g. PyTorch and NumPy. As in the standard formulation, a fold shape is a tuple of values, each describing a *dimension* of the array.

* Within the shape tuple, dimensions are specified in *outermost* to *innermost* order going left to right: for a shape of length `n` describing an array of `n` dimensions, the outermost dimension is `shape[0]`, and the innermost dimension is `shape[n - 1]` (or `shape[-1]`). For example, a matrix with 3 rows of 4 columns each has shape `(3, 4)`.

* The *outward* neighbor of `shape[d]` is `shape[d - 1]`. The *inward* neighbor of `shape[d]` is `shape[d + 1]`. It follows that the outermost dimension has no outward neighbor, and the innermost dimension has no inward neighbor.
  
* `a.transpose(x, y)` swaps the dimensions of `a`'s shape at positions `x` and `y`. For rectangular arrays, this literally means that `a.transpose(x, y).shape[x] == a.shape[y]` and vice versa, with the rest of the shape left unchanged; the effect on a ragged array's shape can be more subtle, as we'll see.
  
* In all cases, given a transposition `a.transpose(x, y)` where `x < y`, `shape[x]` is the *outer dimension* of the transposition and is *transposed inward* to position `y`. Analogously, `shape[y]` is the *inner dimension* of the transposition, and is *transposed outward* to position `x`.
  
* Transposed dimensions `x` (outer) and `y` (inner) are *adjacent* if `x == y - 1`. When transposed dimensions are *nonadjacent*, `x < y - 1` and we call the dimensions `shape[x + 1, ..., y - 1]` the *gap dimensions* of the transposition.
  
* A dimension describes a sequence of *extents*. For example, in the shape `(3, 4)` the inner dimension `shape[1] == 4` is a (compressed) description of the extent of every row (4 columns each). In a fold ragged array, the shape might be `(3, [4, 5, 2])`, describing rows with extents 4, 5 and 2, respectively.
  
* We say a dimension `d` is *non-increasing* if its extents obey the constraint `d[i] <= d[i - 1]` for all valid positions `i` in `d`. For example, the dimension `[3, 2, 2, 1]` is non-increasing; so is the dimension `3`. However, the dimension `[4, 5, 2]` is *increasing* at position 1.

# Rectangular transpose refresher

Recall that to transpose a rectangular array, we swap the transposed dimensions and redistribute the elements accordingly. For example:

In [4]:
print("Array of 3 rows of 4 columns each:")
rows = 3
cols = 4
a = arange(rows, cols)
print("Shape:", a.shape)
print("Values:")
print(a)

row_dim = 0
col_dim = 1
print(f"\nDimensions {row_dim} and {col_dim} transposed (rows and columns swapped):")
t = a.transpose(row_dim, col_dim)
print("Shape", t.shape)
print("Values:")
print(t)

Array of 3 rows of 4 columns each:
Shape: (3, 4)
Values:
[[ 0,  1,  2,  3],
 [ 4,  5,  6,  7],
 [ 8,  9, 10, 11]]

Dimensions 0 and 1 transposed (rows and columns swapped):
Shape (4, 3)
Values:
[[ 0,  4,  8],
 [ 1,  5,  9],
 [ 2,  6, 10],
 [ 3,  7, 11]]


At higher dimensions, transposition can be harder to visualize but operates in exactly the same way. For example, given a 3-dimensional array of 2 sheets, each with 3 rows of 4 columns:

In [5]:
batch = 2
rows = 3
cols = 4
a = arange(batch, rows, cols)
print("Shape:", a.shape)
print("Values:")
print(a)

Shape: (2, 3, 4)
Values:
[[[ 0,  1,  2,  3],
  [ 4,  5,  6,  7],
  [ 8,  9, 10, 11]],

 [[12, 13, 14, 15],
  [16, 17, 18, 19],
  [20, 21, 22, 23]]]


Transposing dimensions 1 and 2 will swap rows and columns within each sheet:

In [6]:
batch_dim = 0
row_dim = 1
col_dim = 2

t = a.transpose(row_dim, col_dim)
print("Shape:", t.shape)
print("Values:")
print(t)

Shape: (2, 4, 3)
Values:
[[[ 0,  4,  8],
  [ 1,  5,  9],
  [ 2,  6, 10],
  [ 3,  7, 11]],

 [[12, 16, 20],
  [13, 17, 21],
  [14, 18, 22],
  [15, 19, 23]]]


Transposing dimensions 0 and 1 will swap sheets and rows - sheet _i_ of the transposed array will contain all rows _i_ from the original:

In [7]:
t = a.transpose(batch_dim, row_dim)
print("Shape:", t.shape)
print("Values:")
print(t)

Shape: (3, 2, 4)
Values:
[[[ 0,  1,  2,  3],
  [12, 13, 14, 15]],

 [[ 4,  5,  6,  7],
  [16, 17, 18, 19]],

 [[ 8,  9, 10, 11],
  [20, 21, 22, 23]]]


Transposing dimensions 0 and 2 will swap sheets and columns - sheet _i_ of the transposed array will contain all columns _i_ from the original:

In [8]:
t = a.transpose(batch_dim, col_dim)
print("Shape:", t.shape)
print("Values:")
print(t)

Shape: (4, 3, 2)
Values:
[[[ 0, 12],
  [ 4, 16],
  [ 8, 20]],

 [[ 1, 13],
  [ 5, 17],
  [ 9, 21]],

 [[ 2, 14],
  [ 6, 18],
  [10, 22]],

 [[ 3, 15],
  [ 7, 19],
  [11, 23]]]


# Generalizing transposition to ragged arrays

What does it mean to transpose a ragged array?

We can begin by noting that since rectangular arrays are a special subset of ragged arrays, our definition must agree exactly with the standard one on this subset (as shown above).

## Transposition Invariants

To extend the definition of transposition to the general, non-rectangular case, we can consider the *invariants* of rectangular transposition, and see how they behave in the presence of raggedness.

1. Transposition is a *total function* on arrays: any two dimensions of an array can be transposed, and the result will also be a well-formed array.
   
2. Transposition redistributes elements so that a given element _A[i, j]_ will be found at _(A^T)[j, i]_ (generalized to any pair of dimensions in an _n_-dimensional array)

A consequence of these invariants which can be useful in guiding intuition is

3. Transposition is *invertible*: transposing an array twice (on the same dimensions) will produce the original array.

## Shear

For most useful combinations of ragged shapes and transposed dimensions, these invariants are easily preserved.

However, a subset of ragged transpositions will cause a distortion in the result - fold uses the term *shear* for this distortion, due to its similarity to the shearing transformation of rectangular matrices. When shearing occurs, invariants #2 and (thus) #3 above no longer hold.

For intuition, consider the following 2-dimensional ragged array and its transposition:
```
[[0],
 [1, 2],
 [3],
 [4, 5]]
->
[[0, 1, 3, 4],
 [2, 5]]
```
Note how the elements from the ragged second column of the original array have been collected into a dense row in the transpose, breaking invariant #2. In a nutshell, a ragged array can shear under transposition due to *preservation of density*: ragged arrays are by definition dense - their shapes only model variation in *extent* along a dimension, not variation in *placement*. Avoiding shear in this example would require explicit placement of elements at sparse offsets, which purely ragged arrays have no way to express.

(Aside: fold of course provides general tools for constructing sparse arrays, as *indexed compositions* of dense ragged arrays. See the "Digression: Sparsity" section at the end of this notebook, and notes elsewhere in this repo.)

Next consider the following 2-dimensional ragged array, its transpose, and its double transpose:
```
[[0, 1, 3, 4],
 [2, 5]]
->
[[0, 2],
 [1, 5],
 [3],
 [4]]
->
[[0, 1, 3, 4],
 [2, 5]]
```
Here shearing does not occur and transposition remains invertible. This is because the extents of the ragged dimension are *non-increasing*: the ragged dimension is `[4, 2]` in the first case and `[2, 2, 1, 1]` in the second case.

(Note that our starting array in this example is just the result of the transpose in the previous example. Transposition in the presence of shear is *idempotent*.)

With this intuition in mind, we'll now classify ragged transpositions more rigorously.



## Classifying the action of transposition on ragged dimensions

To more precisely understand the effect of transposition on ragged *shapes* (including the circumstances under which shear can occur), we can classify the effect of transposition on individual ragged *dimensions*, based on where in the shape they appear relative to the dimensions being transposed.

We'll enumerate the possible relationships here, followed later by examples of each.

1. A ragged dimension *outward* of the transposed dimensions is unaffected by the transposition.
   
2. A ragged dimension *inward* of the transposed dimensions is itself transposed.
   
3. A ragged dimension being *transposed inward* (that is, a ragged dimension that is the outer dimension of a transpose) is "spread": its elements are expanded by those of the inner dimension being transposed outward, combined with those of any "gap" dimensions between the transposed dimensions.

4. A ragged dimension being *transposed outward* (that is, a ragged dimension that is the inner dimension of a transpose) will shear, unless its extents are *non-increasing* within the cells formed by each of the outer dimensions it is being transposed across. The shearing phenomenon is illustrated for simple examples above; for intuition on more involved examples, see the section on this case (#4) below.
   
5. A ragged "gap" dimension (between the two transposed dimensions) will shear, unless its extents are *non-increasing* within the cells formed by each of the dimensions between it and (inclusively) the outer dimension being transposed. (This case is closely related to #4.)

Note that since ragged dimensions may appear anywhere in a ragged shape (except, trivially, its outermost dimension) any combination of these relationships may be in play during a particular transposition.

Below are examples of transpositions demonstrating each of these relationships and their effects. But first, let's briefly zoom out to the bigger picture.

## Defining transpose() in a ragged array API

Given the possibility of shear, we essentially have these classes of options for how we define a transform function on ragged arrays:

1. Make transpose a *partial* function, defined only on non-shearing transpositions.
   
2. Relax invariant #2 above, making transposition an operation that may leave elements in unexpected places, and is not always invertible (or, for that matter, always linear).
   
3. Define transposition on a more expressive domain (essentially, masked arrays) where it can remain total, and provide conversions (total in the Array -> MaskedArray direction, partial in the MaskedArray -> Array direction).

Currently fold opts for a version of #1, providing a guard method  `transpose_will_shear` and raising an exception on shearing transpositions. However, this design decision has no downstream dependencies beyond the implementation of transposition itself. Some concrete alternatives include:

* lift `transpose()` to `MaskedArray`, restoring totality but complicating user code with conversions (and still requiring user guards)

* have `transpose()` internally lift to masked arrays only when necessary, and return `Union[Array, MaskedArray]`

* make `transpose()` total over arrays by allowing shear, significantly weakening its invariants

* leave `transpose()` partial, but have it return `Option[Array]` instead of raising an error

# Case Examples

Following are examples that demonstrate the different cases described in **Classifying the action of transposition on ragged dimensions** above.


## Case 1: a ragged dimension *outward* of the transposed dimensions

This case is straightforward: any dimension (ragged or not) that is outward of a transposition - that is part of the frame within which cells are transposed - is undisturbed by the transposition.

(For those unfamiliar with the terminology, any `(n+m)`-dimensional array can be viewed as an `n`-dimensional frame containing `m`-dimensional cells, for any `n` and `m` that sum to the total number of array dimensions.)

Here's a simple example in which a 2-dimensional frame (with one of the dimensions ragged) contains 2-dimensional cells:

In [9]:
batch_size = 2
seq_len = [1, 2]
item_height = 2
item_width = 3

print("Ragged sequence lengths:")
a = arange(batch_size, seq_len, item_height, item_width)
print("Shape:", a.shape)
print(a)

Ragged sequence lengths:
Shape: (2, [1, 2], 2, 3)
[[[[ 0,  1,  2],
   [ 3,  4,  5]]],


 [[[ 6,  7,  8],
   [ 9, 10, 11]],

  [[12, 13, 14],
   [15, 16, 17]]]]


When we transpose the cells, the frame is unaffected:

In [10]:
print("Item height and width transposed, sequence length unaffected:")
t = a.transpose(item_height, item_width)
print("Shape:", t.shape)
print(t)

Item height and width transposed, sequence length unaffected:
Shape: (2, [1, 2], 3, 2)
[[[[ 0,  3],
   [ 1,  4],
   [ 2,  5]]],


 [[[ 6,  9],
   [ 7, 10],
   [ 8, 11]],

  [[12, 15],
   [13, 16],
   [14, 17]]]]


## Case 2: a ragged dimension *inward* of the transposed dimensions

To build intuition for this case, consider an array containing ragged 1-dimensional cells within a 2-dimensional outer frame:

In [11]:
batch_size = 2
seq_len = 3 # each sequence has 3 items
seq_item_widths = [1, 2, 3] # sequence items will repeat this pattern

print("Batch of ragged sequences:")
a = arange(batch_size, seq_len, seq_item_widths)
print("Shape:", a.shape)
print(a)

Batch of ragged sequences:
Shape: (2, 3, Repeat([1, 2, 3], 2))
[[[ 0],
  [ 1,  2],
  [ 3,  4,  5]],

 [[ 6],
  [ 7,  8],
  [ 9, 10, 11]]]


Transposing the outer frame will also transpose the cells, resulting in a transposition of the dimension that describes the cell extents:

In [12]:
batch_dim = 0
seq_dim = 1

print("Outer frame (batch and sequence dimensions) transposed, inner ragged shape also transposed:")
t = a.transpose(batch_dim, seq_dim)
print("Shape:", t.shape)
print(t)

Outer frame (batch and sequence dimensions) transposed, inner ragged shape also transposed:
Shape: (3, 2, [1, 1, 2, 2, 3, 3])
[[[ 0],
  [ 6]],

 [[ 1,  2],
  [ 7,  8]],

 [[ 3,  4,  5],
  [ 9, 10, 11]]]


Of course, *this same reordering* occurs in rectangular arrays, but is invisible at the shape level due to the regularity of rectangular dimensions.

### Aside: the fold `Repeat` dimension

Fold provides a small set of data structures for representing ragged dimensions that exhibit different kinds of regularity. `Repeat` is one such structure: the ragged dimension `Repeat([1, 2, 3], 2)` is simply the following sequence, encoded in packed form:

In [13]:
repeating_dim = Repeat([1, 2, 3], 2)
print([i for i in repeating_dim])

[1, 2, 3, 1, 2, 3]


## Case 3: a ragged dimension *transposed inward*

When the outer dimension of a transpose is swapped with the inner dimension, we say that the outer dimension is *transposed inward*.

When the outer dimension is ragged, the transposition will "spread" it, in the sense that each of its extents will be repeated by an amount given by the corresponding extent of new (post-transpose) outer dimension, *multiplied* by the extents of any "gap" dimensions appearing between the transposed dimensions.

#### Simple example (no gap dimensions)

First a simple example, in which the transposed dimensions are adjacent (so there is no gap). Here we have a 3-dimensional array with a ragged middle dimension, and we want to transpose the inner dimensions:

In [14]:
batch_size = 2
seq_lens = [2, 3]
item_width = 3

print("Batch of variable-length sequences of fixed-length items:")
a = arange(batch_size, seq_lens, item_width)
print("Shape:", a.shape)
print(a)

Batch of variable-length sequences of fixed-length items:
Shape: (2, [2, 3], 3)
[[[ 0,  1,  2],
  [ 3,  4,  5]],

 [[ 6,  7,  8],
  [ 9, 10, 11],
  [12, 13, 14]]]


Intuitively, when we transpose the sequence and item dimensions (1 and 2) of this array, we are swapping rows and columns within each sheet. In doing so, we transform the ragged shape of the rows - the different number of rows in each sheet - into a ragged shape of *sequences of columns*.

That is, as the shape travels inward, the number of things each of its extents describes is multiplied:

In [15]:
seq_dim = 1
item_dim = 2

print("Sequence and item dimensions (1 and 2) transposed. Sequence dimension is transposed outward and spread:")
t = a.transpose(seq_dim, item_dim)
print("Shape:", t.shape)
print(t)

Sequence and item dimensions (1 and 2) transposed. Sequence dimension is transposed outward and spread:
Shape: (2, 3, Runs([2, 3], 3))
[[[ 0,  3],
  [ 1,  4],
  [ 2,  5]],

 [[ 6,  9, 12],
  [ 7, 10, 13],
  [ 8, 11, 14]]]


### Aside: the fold `Runs` dimension

`Runs` is another of the fold data structures used to encode ragged dimensions that exhibit regularity - in this case, the regularity consists of the sequence repeating each value a certain number of times before switching to the next value, a pattern can be efficiently represented using run-length encoding.

Here, the ragged dimension `Runs([2, 4], 3)` is simply the following sequence encoded in run-length encoded form:

In [16]:
run_length_encoded_dim = Runs([2, 4], 3)
print([i for i in run_length_encoded_dim])

[2, 2, 2, 4, 4, 4]


### The SDPA transpose

A canonical example of transposing a ragged dimension inward is the one used in Scaled Dot Product Attention to bring the attention head dimension outward of the token sequence dimension.

Intuitively, when we transpose dimension 1 (`seq_len`) with dimension 2 (`num_heads`), we're transforming
* a batch of sequences divided into (`num_head` x `head_dim`) matrices

into
* a batch of attention heads divided into (`seq_len` x `head_dim`) matrices:

In [23]:
batch_size = 2
seq_lens = [4, 5]
num_heads = 2
emb_width = 3

a = arange(batch_size, seq_lens, num_heads, emb_width)

print(f"{batch_size} sequences of lengths {seq_lens}.")
print(f"Each sequence element contains {num_heads} embeddings (one per attention head) of dimension {emb_width} (embedding width)")
print("\nShape:", a.shape)
print(a)

2 sequences of lengths [4, 5].
Each sequence element contains 2 embeddings (one per attention head) of dimension 3 (embedding width)

Shape: (2, [4, 5], 2, 3)
[[[[ 0,  1,  2],
   [ 3,  4,  5]],

  [[ 6,  7,  8],
   [ 9, 10, 11]],

  [[12, 13, 14],
   [15, 16, 17]],

  [[18, 19, 20],
   [21, 22, 23]]],


 [[[24, 25, 26],
   [27, 28, 29]],

  [[30, 31, 32],
   [33, 34, 35]],

  [[36, 37, 38],
   [39, 40, 41]],

  [[42, 43, 44],
   [45, 46, 47]],

  [[48, 49, 50],
   [51, 52, 53]]]]


In [25]:
seq_dim = 1
head_dim = 2

t = a.transpose(seq_dim, head_dim)

print(f"{batch_size} batches of {num_heads} attention heads each.")
print(f"Within each batch, attention heads contain {seq_lens} embeddings (respectively) of dimension {emb_width} (embedding width)")
print("\nTransposed shape:", t.shape)
print(t)

2 batches of 2 attention heads each.
Within each batch, attention heads contain [4, 5] embeddings (respectively) of dimension 3 (embedding width)

Transposed shape: (2, 2, Runs([4, 5], 2), 3)
[[[[ 0,  1,  2],
   [ 6,  7,  8],
   [12, 13, 14],
   [18, 19, 20]],

  [[ 3,  4,  5],
   [ 9, 10, 11],
   [15, 16, 17],
   [21, 22, 23]]],


 [[[24, 25, 26],
   [30, 31, 32],
   [36, 37, 38],
   [42, 43, 44],
   [48, 49, 50]],

  [[27, 28, 29],
   [33, 34, 35],
   [39, 40, 41],
   [45, 46, 47],
   [51, 52, 53]]]]



### Case 3 example with gap dimensions

Now let's look at an example involving gap dimensions, to build intuition for the multiplicative effect they have on the spreading operation.

Our array for this example is 4-dimensional, with a ragged dimension 1 - we can think of this shape as "unequal-size batches of sheets." We aim to transpose dimensions 1 and 3 ("swapping sheets for columns").

In [26]:
batch_size = 2
seq_lens = [2, 3]
num_heads = 2
emb_width = 3

print("4D array with ragged dimension 1:")
a = arange(batch_size, seq_lens, num_heads, emb_width)
print("Shape:", a.shape)
print(a)

4D array with ragged dimension 1:
Shape: (2, [2, 3], 2, 3)
[[[[ 0,  1,  2],
   [ 3,  4,  5]],

  [[ 6,  7,  8],
   [ 9, 10, 11]]],


 [[[12, 13, 14],
   [15, 16, 17]],

  [[18, 19, 20],
   [21, 22, 23]],

  [[24, 25, 26],
   [27, 28, 29]]]]


Now when we transpose dimensions 1 and 3, dimension 1 will be spread by *both* dimensions 2 and 3. By the same intuition as in our gapless example, what has happened is that the ragged batch dimension must now describe batched sequences of columns:

In [27]:
seq_dim = 1
emb_dim = 3

print("Dimensions 1 and 3 transposed, dimension 1 transposed outward to 3 and spread:")
t = a.transpose(seq_dim, emb_dim)
print("Shape:", t.shape)
print(t)

Dimensions 1 and 3 transposed, dimension 1 transposed outward to 3 and spread:
Shape: (2, 3, 2, Runs([2, 3], 6))
[[[[ 0,  6],
   [ 3,  9]],

  [[ 1,  7],
   [ 4, 10]],

  [[ 2,  8],
   [ 5, 11]]],


 [[[12, 18, 24],
   [15, 21, 27]],

  [[13, 19, 25],
   [16, 22, 28]],

  [[14, 20, 26],
   [17, 23, 29]]]]


## Case 4: A ragged dimension being *transposed outward*

A ragged *inner* dimension *transposed outward* is safe from shear as long as its extents are *non-increasing* (in effect, sorted in descending order) within the cells formed by the outer dimension(s) it is being transposed across.

We went through a simple example of this case earlier, when we introduced shear. Here we'll illustrate its behavior in more general situations.

To start, here's an array with a slightly more involved shape - a batch of unequal-sized rectangles. Dimensions 1 and 2 are both ragged: dimension 1 tells us the (varying) height of each rectangle, and dimension 2 tells us width of each row (constant within rectangles, but varying from one rectangle to another):

In [28]:
num_rects = 3
rect_heights = [3, 2, 3]
row_widths = Runs([4, 3, 4], [3, 2, 3]) # each rect has rows of identical width

print("3D array with ragged dimensions 1 and 2 (batch of unequal-shape rectangles):")
a = arange(num_rects, rect_heights, row_widths)
print("Shape:", a.shape)
print("Shape unpacked:", a.shape.unpack())
print(a)

3D array with ragged dimensions 1 and 2 (batch of unequal-shape rectangles):
Shape: (3, [3, 2, 3], Runs([4, 3, 4], [3, 2, 3]))
Shape unpacked: ([3], [3, 2, 3], [4, 4, 4, 3, 3, 4, 4, 4])
[[[ 0,  1,  2,  3],
  [ 4,  5,  6,  7],
  [ 8,  9, 10, 11]],

 [[12, 13, 14],
  [15, 16, 17]],

 [[18, 19, 20, 21],
  [22, 23, 24, 25],
  [26, 27, 28, 29]]]


Transposing dimensions 1 and 2 of this array will transpose each rectangle in place.

Note that in this case *both* diemnsions being transposed are ragged: our focus here is on what happens to the inner ragged dimension as it is transposed outward, but we can also observe multiple ragged dimensions being involved in different ways in the same transposition.

In [29]:
height_dim = 1
width_dim = 2

print("Dimensions 1 and 2 transposed:")
t = a.transpose(height_dim, width_dim)
print("Shape:", t.shape)
print(t)

Dimensions 1 and 2 transposed:
Shape: (3, [4, 3, 4], Runs([3, 2, 3], [4, 3, 4]))
[[[ 0,  4,  8],
  [ 1,  5,  9],
  [ 2,  6, 10],
  [ 3,  7, 11]],

 [[12, 15],
  [13, 16],
  [14, 17]],

 [[18, 22, 26],
  [19, 23, 27],
  [20, 24, 28],
  [21, 25, 29]]]


This transposition avoids shear because its ragged inner dimension is non-increasing within the cells of the outer dimension it is being transposed across (in fact, within those cells it's rectangular).

For a more flagrantly ragged example, consider the following array and its transpose. Here the batch elements are no longer rectangular, but they maintain the *non-increasing* property that within each cell, no extent is greater than its predecessor.

In [30]:
batch_size = 3
seq_lens = [3, 2, 3]
item_widths = [3, 2, 1, 3, 1, 2, 1, 1] # items within a sequence have different widths

print("\nArray of variable length sequences with variable width items in each sequence (ragged dimensions 1 and 2):")
a = arange(batch_size, seq_lens, item_widths)
print("Shape:", a.shape)
print(a)

seq_dim = 1
item_dim = 2
print("\nSequence and item dimensions (1 and 2) transposed, rows are now item elements at each width position")
t = a.transpose(seq_dim, item_dim)
print("Shape:", t.shape)
print(t)


Array of variable length sequences with variable width items in each sequence (ragged dimensions 1 and 2):
Shape: (3, [3, 2, 3], [3, 2, 1, 3, 1, 2, 1, 1])
[[[ 0,  1,  2],
  [ 3,  4],
  [ 5]],

 [[ 6,  7,  8],
  [ 9]],

 [[10, 11],
  [12],
  [13]]]

Sequence and item dimensions (1 and 2) transposed, rows are now item elements at each width position
Shape: (3, Runs([3, 2], [2, 1]), [3, 2, 1, 2, 1, 1, 3, 1])
[[[ 0,  3,  5],
  [ 1,  4],
  [ 2]],

 [[ 6,  9],
  [ 7],
  [ 8]],

 [[10, 12, 13],
  [11]]]


## Case 5: A ragged dimension between two transposed dimensions

A ragged "gap" dimension (one that lies between the two dimensions being transposed) is safe from shear as long as its extents are non-increasing within the cells formed by each of the dimensions between it and (inclusively) the outer dimension being transposed.

This case is closely related to the previous one.

In this example, dimensions 0 and 1 define two cells of 3 matrices each, Dimension 2 gives the height of each matrix, and has non-increasing extents within each of these cells. (Note that the shape does *not* repeat identically within each cell.)

In [31]:
batch_size = 2
seq_len = 3
item_heights = [6, 5, 4, 3, 2, 1]
item_width = 2

a = arange(batch_size, seq_len, item_heights, item_width)
print("Shape:", a.shape)
print(a)

Shape: (2, 3, [6, 5, 4, 3, 2, 1], 2)
[[[[ 0,  1],
   [ 2,  3],
   [ 4,  5],
   [ 6,  7],
   [ 8,  9],
   [10, 11]],

  [[12, 13],
   [14, 15],
   [16, 17],
   [18, 19],
   [20, 21]],

  [[22, 23],
   [24, 25],
   [26, 27],
   [28, 29]]],


 [[[30, 31],
   [32, 33],
   [34, 35]],

  [[36, 37],
   [38, 39]],

  [[40, 41]]]]


We can transpose dimensions 1 and 3 (on either side of ragged dimension 2) without shearing, because dimension 2 is non-increasing within the cells of dimension 1:

In [32]:
seq_dim = 1
item_width_dim = 3

t = a.transpose(seq_dim, item_width_dim)
print("Shape:", t.shape)
print(t)

Shape: (2, 2, Runs([6, 3], [2, 2]), Runs([3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1], [4, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1]))
[[[[ 0, 12, 22],
   [ 2, 14, 24],
   [ 4, 16, 26],
   [ 6, 18, 28],
   [ 8, 20],
   [10]],

  [[ 1, 13, 23],
   [ 3, 15, 25],
   [ 5, 17, 27],
   [ 7, 19, 29],
   [ 9, 21],
   [11]]],


 [[[30, 36, 40],
   [32, 38],
   [34]],

  [[31, 37, 41],
   [33, 39],
   [35]]]]


This example helps build intuition for *why* the non-increasing property prevents shear: here, within each cell of 3 matrices, the transposition takes columns stacked vertically and restacks them horizontally, left to right. For this to work without shearing, the columns need to be non-increasing in height.

#### The `transpose_will_shear()` guard

In fold, `transpose()` will raise an exception if the transposition would shear. To guard against this, `transpose_will_shear()` can be called to check for shearing. Using our original shearing example:

In [33]:
rows = 4
cols = [1, 2, 1, 2]

height_dim = 0
width_dim = 1

a = arange(rows, cols)
print("Shape:", a.shape)
print(a)
print(f"\n{a.transpose_will_shear(height_dim, width_dim)=}")

Shape: (4, [1, 2, 1, 2])
[[0],
 [1, 2],
 [3],
 [4, 5]]

a.transpose_will_shear(height_dim, width_dim)=True


However as noted initially, this API is a design choice, not a fundamental constraint.

# Appendix: Sparsity

As noted above, ragged arrays are definitionally dense, and lack the means to express sparsity. This is essential to a properly factored array model: raggedness is a fundamentally simpler property than sparsity, and the rules for ragged arrays should not be encumbered by this extra machinery.

Instead, sparsity should be modeled in terms of *composition* of ragged arrays. In fold, indexes are expressive enough to model the sparse scattering of elements into a backing array, enabling sparse arrays to be neatly expressed as deferred ("lazy") indexed assignments.

Stepping back: in fold arrays can be composed both eagerly/mutatively, via direct indexed assignment, and lazily/functionally, via what fold calls *overlays*. The observable results of an indexed assignment and its equivalent overlay are identical - the only difference is evaluation:

* An indexed assignment is evaluated immediately, modifying the destination array.

* An overlay is a lazy structure that holds references to both source and destination arrays, along with the index (actually a view) locating source elements within the destination. An overlay's __setitem__ and __getitem__ methods mimic the result of the corresponding indexed assignment.

As should be clear, traditional sparse representations are overlays:

* the destination array is a field of zero elements. In purpose-built sparse data structures this is normally implicit or denoted only by shape metadata and possibly a fill value; in fold the array is explicit (since an overlay can have any array or view as its destination, just like an indexed assignment), but use of zero striding essentially reduces any constant-value array to its shape and a fill value, in terms of storage cost.

* the source array is the "values" structure, using standard sparse representation terminology

* the view that locates source elements within the destination (fold calls this the image of the overlay) is the "index" structure, in sparse parlance. We can express the various traditional sparse formats using different varieties of raggedness in the image.

As a quick example, let's revisit the ragged array example used above to demonstrate *shear* (replacing arange with random for readablity):

In [34]:
a = rand(4, [1, 2, 1, 2])
print("Ragged:\n")
print(a)

Ragged:

[[0.5912],
 [0.7411, 0.5897],
 [0.1045],
 [0.2398, 0.3420]]


Before using an overlay to model a sparse structure containing these values, let's create a dense representation of the same array by scattering the elements of `a` into a backing array filled with zeros. We can do this in fold using ordinary indexed assignment.

Here, `a.image()` produces a tuple of (advanced) index values, one per dimension, which when used together index the entire array `a`.

In [35]:
for i, ix in enumerate(a.image()):
  print(f"a.image()[{i}]:")
  print(ix)
  print()
print(f"a[a.image()] == a:", a[a.image()] == a)

a.image()[0]:
[[0],
 [1, 1],
 [2],
 [3, 3]]

a.image()[1]:
[[0],
 [0, 1],
 [0],
 [0, 1]]

a[a.image()] == a: True


Using this index we can project `a` onto a rectangular backing array:

In [36]:
scattered = zeros(4, 2)
print("Before scattering assignment:")
print(scattered)
scattered[a.image()] = a
print("\nAfter scattering assignment:")
print(scattered)

Before scattering assignment:
[[0.0000, 0.0000],
 [0.0000, 0.0000],
 [0.0000, 0.0000],
 [0.0000, 0.0000]]

After scattering assignment:
[[0.5912, 0.0000],
 [0.7411, 0.5897],
 [0.1045, 0.0000],
 [0.2398, 0.3420]]


Now let's create an overlay to *lazily* represent the result of this scattering operation. It will turn out that the components of an overlay correspond directly to the substructures of a standard sparse representation.

(Note that laziness means we can use zero strides to reduce our "fill" array data to a single scalar.)

In [37]:
fill = zeros().expand(4, 2)
print("Fill array:")
print(fill)
print("\nFill array data:", fill.data)
print("Fill array shape:", fill.view.shape)
print("Fill array strides:", fill.view.strides)

ov = fill.overlay[a.image()](a)
print("\nOverlay of `a` onto fill array:")
print(ov)

print("\nOverlay and scattered array are observationally equivalent:")
print("ov.tolist() == scattered.tolist():", ov.tolist() == scattered.tolist())

Fill array:
[[0.0000, 0.0000],
 [0.0000, 0.0000],
 [0.0000, 0.0000],
 [0.0000, 0.0000]]

Fill array data: tensor([0.])
Fill array shape: (4, 2)
Fill array strides: (0, 0)

Overlay of `a` onto fill array:
[[0.5912, 0.0000],
 [0.7411, 0.5897],
 [0.1045, 0.0000],
 [0.2398, 0.3420]]

Overlay and scattered array are observationally equivalent:
ov.tolist() == scattered.tolist(): True


The source and destination components of the overlay are unmodified: it simply holds references to the arrays specified in the overlay construction expression:

In [38]:
print(f"{ov.dest is fill=}")
print(f"{ov.src is a=}")

ov.dest is fill=True
ov.src is a=True


To relate source and destination, the overlay builds an *image*, an indexing view whose shape is that of the source array, and whose strides specify the destination locations of the source array's elements.

In [39]:
print("ov.image.shape:", ov.image.shape)
print("ov.image.strides:", ov.image.strides)

ov.image.shape: (4, [1, 2, 1, 2])
ov.image.strides: (2, [2, 1, 1, 2, 1, 1])


For intuition, note that the overlay's image is exactly the view we would construct to *recover* our original ragged array from the padded rectangular array `r` we created earlier:

In [40]:
print("a recovered from scattered:\n")
print(Array(scattered.data, ov.image))


a recovered from scattered:

[[0.5912],
 [0.7411, 0.5897],
 [0.1045],
 [0.2398, 0.3420]]
